In [4]:
import spectrum
from scipy.interpolate import interp1d
from astropy.io import ascii, fits
field = "COSMOSC"
ifu = "025"
Apix = 0.5**2.
ftcal = "../specphot/mean_cal_{}.txt".format(field)
#ftcal = "1237651753997238489_cal.txt"
fcubein = "../data/outcube_{}_{}.fits.gz".format(field,ifu)
fcubeout = "../data/coutcube_{}_{}.fits.gz".format(field,ifu)
tcal = ascii.read(ftcal, format="fixed_width")

cal_interp = interp1d(tcal["wl[A]"], tcal["cal[erg/s/cm^2/A/cnt]"], kind='linear', bounds_error=False,fill_value="extrapolate")


s = spectrum.readSpectrum(fcubein)

ww = s.grid()
for i,w in enumerate(ww):
    s.data[i] = s.data[i] * cal_interp(w)/Apix * 1e17

h = fits.PrimaryHDU(data=s.data, header=s.hdu.header)
h.header['BUNITS'] = '10x-17 erg/s/cm^2/A/arcsec^2'
h.writeto(fcubeout, overwrite=True)
print("Done")